In [76]:
import pandas as pd

# read the data
url = "https://raw.githubusercontent.com/fja05680/sp500/master/S%26P%20500%20Historical%20Components%20%26%20Changes(04-16-2023).csv"
df = pd.read_csv(url)

# split the tickers into separate rows
df = df.assign(tickers=df.tickers.str.split(","))
df = df.explode("tickers")

# create a count column for pivot table
df['count'] = 1

# create the panel data
panel_data = df.pivot_table(index='date', columns='tickers', values='count', aggfunc='sum')
panel_data.fillna(0, inplace=True)

# melt the data to create a "long" format
melted_data = pd.melt(panel_data.reset_index(), id_vars='date', var_name='tickers', value_name='value')

# select rows where value equal 1
melted_data = melted_data[melted_data["value"] == 1]

# drop the "value" column
melted_data.drop('value', axis=1, inplace=True)

# convert the date column to datetime format
melted_data['date'] = pd.to_datetime(melted_data['date'])

# group the tickers by date
grouped_data = melted_data.groupby('date')['tickers'].apply(list)

# select the row of first date of each year
first_dates = grouped_data.groupby(grouped_data.index.year).apply(lambda x: x.iloc[0])

# transpose the resulting data
transposed_data = pd.DataFrame(first_dates.tolist(), index=first_dates.index)
transposed_data = transposed_data.transpose()

# display the resulting data transposed
print(transposed_data)

# save csv file
transposed_data.to_csv('sp500_constituents.csv')

date   1996   1997   1998   1999   2000   2001   2002   2003  2004  2005  ...  \
0       AAL    AAL  AAMRQ  AAMRQ   AABA      A      A      A     A     A  ...   
1     AAMRQ  AAMRQ   AAPL   AAPL  AAMRQ   AABA   AABA   AABA  AABA  AABA  ...   
2      AAPL   AAPL    ABI    ABI   AAPL  AAMRQ  AAMRQ  AAMRQ  AAPL  AAPL  ...   
3       ABI    ABI    ABS    ABS    ABI   AAPL   AAPL   AAPL   ABC   ABC  ...   
4       ABS    ABS    ABT    ABT    ABS    ABI    ABC    ABC   ABI   ABI  ...   
..      ...    ...    ...    ...    ...    ...    ...    ...   ...   ...  ...   
501    None   None   None   None   None   None   None   None  None  None  ...   
502    None   None   None   None   None   None   None   None  None  None  ...   
503    None   None   None   None   None   None   None   None  None  None  ...   
504    None   None   None   None   None   None   None   None  None  None  ...   
505    None   None   None   None   None   None   None   None  None  None  ...   

date  2014  2015  2016  201

In [77]:
all_tickers = pd.read_csv("sp500_constituents.csv")
all_tickers.head()

,Unnamed: 0,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,0,AAL,AAL,AAMRQ,AAMRQ,AABA,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
1,1,AAMRQ,AAMRQ,AAPL,AAPL,AAMRQ,AABA,AABA,AABA,AABA,...,AABA,AABA,AABA,AABA,AAL,AAL,AAL,AAL,AAL,AAL
2,2,AAPL,AAPL,ABI,ABI,AAPL,AAMRQ,AAMRQ,AAMRQ,AAPL,...,AAPL,AAPL,AAL,AAL,AAP,AAP,AAP,AAP,AAP,AAP
3,3,ABI,ABI,ABS,ABS,ABI,AAPL,AAPL,AAPL,ABC,...,ABBV,ABBV,AAP,AAP,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
4,4,ABS,ABS,ABT,ABT,ABS,ABI,ABC,ABC,ABI,...,ABC,ABC,AAPL,AAPL,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV
